In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak

In [2]:
filename = 'pionplus_1k.npy'
file = open(filename, 'rb')
mc_truth = np.load(file,allow_pickle=True) #Added Generator E and Angle
data = np.load(file,allow_pickle=True)

test_file = open('test.npy', 'rb')
test_data = np.load(test_file,allow_pickle=True)

In [3]:
print("----------------- ORIGINAL test.npy -------------------")
print('number of images in event', len(test_data[0]))
#print(test_data[0][0])

----------------- ORIGINAL test.npy -------------------
number of images in event 400


In [4]:
print("--------------- NEW .npy ----------------")
print('number of images in event', len(data[0]))
#print(data[0][0])
print(np.shape(data))

--------------- NEW .npy ----------------
number of images in event 400
(1000, 400)


In [5]:
mc_truth.item().get('true_energy')

20.007061

In [6]:
#============ Original ============================
X = []
Y = []
with open('test.npy', 'rb') as f:#wrong file for now
#with open(filename, 'rb') as f:
    data = np.load(f,allow_pickle=True)
    ievt = 0
    print('number of images in event', len(data[ievt]))
    ## Here we loop over all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
    ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)
    for im in range(len(data[ievt])):
        event = np.c_[data[ievt][im]['HCAL1_x'][0],data[ievt][im]['HCAL1_y'][0],data[ievt][im]['HCAL1_E'][0]]
        depths = np.array([2*np.ones(len(data[ievt][im]['HCAL1_x'][0]))]).flatten()
        if (len(data[ievt][im]['HCAL2_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL2_x'][0],data[ievt][im]['HCAL2_y'][0],data[ievt][im]['HCAL2_E'][0]]])
            depths = np.concatenate([depths,np.array([3*np.ones(len(data[ievt][im]['HCAL2_x'][0]))]).flatten()])
        if (len(data[ievt][im]['HCAL3_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL3_x'][0],data[ievt][im]['HCAL3_y'][0],data[ievt][im]['HCAL3_E'][0]]])
            depths = np.concatenate([depths,np.array([4*np.ones(len(data[ievt][im]['HCAL3_x'][0]))]).flatten()])
        event = np.concatenate([event,np.c_[data[ievt][im]['ECAL_x'],data[ievt][im]['ECAL_y'],data[ievt][im]['ECAL_E']]])
        depths = np.concatenate([depths,np.array([1*np.ones(len(data[ievt][im]['ECAL_x']))]).flatten()])
        event = np.insert(event, 3, depths,axis=1)
        event = np.insert(event, 4, data[ievt][im]['boundary'][0]*np.ones(len(event)),axis=1) #better to make these global features, but I did not want to download the latest energyflow package
        event = np.insert(event, 5, data[ievt][im]['boundary'][1]*np.ones(len(event)),axis=1)
        X += [event]
        trueenergy = 20.007061 #Miguel, please add this!
        Y += [trueenergy]
X = np.array(X)
Y = np.array(Y)
print(np.shape(X))
print(np.shape(Y))

number of images in event 400
(400, 96, 6)
(400,)


In [7]:
def awk_to_numpy_padded(awk_array,fill_val,ncell_max=1200,cell_axis=2,clip=True):

    none_padded = ak.pad_none(ak.ArrayBuilder.snapshot(awk_array), ncell_max, axis=cell_axis, clip=True)
    none_to_val = ak.fill_none(none_padded,fill_val,axis=cell_axis)
    array = np.squeeze(ak.to_numpy(none_to_val))

    return(array)

ncell_max = 1200 #Root file indicates avg. of ~270 per ECal and HCal each. Safety X 2
cell_axis = 2
n_cell_variables = 6
fill_val = np.zeros(n_cell_variables) #for zero paddin awkward array
#fill_val = np.full([6], np.nan)
#print(fill_val)

In [ ]:
X_awk = ak.ArrayBuilder()
X_awk.begin_list()
Y = []

with open(filename, 'rb') as f:
#with open("test.npy", 'rb') as f:
    mc_truth = np.load(f,allow_pickle=True)
    data = np.load(f,allow_pickle=True)
    
    print('N Events:', len(data),'N Images per Event:',len(data[0]))
    
    #ievt=4
    #if not(ievt==999):
    for ievt in range(0,len(data)):
    #for ievt in range(0,1):
        
        ## Here we loop over all events and then all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
        ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)

        for im in range(len(data[ievt])):
            #FIXME: assumes ECAL is always hit...
            event = np.c_[data[ievt][im]['ECAL_x'],data[ievt][im]['ECAL_y'],data[ievt][im]['ECAL_E']]
            depths = np.array([1*np.ones(len(data[ievt][im]['ECAL_x']))]).flatten()
            
            if (ievt == im == 0):
                print("ECAL X:", np.shape(data[ievt][im]['ECAL_x']), "+ ECAL Y", np.shape(data[ievt][im]['ECAL_y']),
                  "+ ECAL E", np.shape(data[ievt][im]['ECAL_E']),"=",np.shape(event), "[using np.c_]")

            if (len(data[ievt][im]['HCAL1_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL1_x'][0],data[ievt][im]['HCAL1_y'][0],data[ievt][im]['HCAL1_E'][0]]])
                depths = np.concatenate([depths,np.array([2*np.ones(len(data[ievt][im]['HCAL1_x'][0]))]).flatten()])
            
            if (len(data[ievt][im]['HCAL2_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL2_x'][0],data[ievt][im]['HCAL2_y'][0],data[ievt][im]['HCAL2_E'][0]]])
                depths = np.concatenate([depths,np.array([3*np.ones(len(data[ievt][im]['HCAL2_x'][0]))]).flatten()])
                
            if (len(data[ievt][im]['HCAL3_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL3_x'][0],data[ievt][im]['HCAL3_y'][0],data[ievt][im]['HCAL3_E'][0]]])
                depths = np.concatenate([depths,np.array([4*np.ones(len(data[ievt][im]['HCAL3_x'][0]))]).flatten()])
            
            if (ievt == im == 0):
                print("HCAL X:", np.shape(data[ievt][im]['HCAL3_x']), "+ HCAL Y", np.shape(data[ievt][im]['HCAL3_y']), 
                      "+ HCAL E",np.shape(data[ievt][im]['HCAL3_E']),"= ", "-> np.append to ECAL ->",np.shape(event))
            
            event = np.insert(event, 3, depths,axis=1)
            event = np.insert(event, 4, data[ievt][im]['boundary'][0]*np.ones(len(event)),axis=1)
            #FIXME: use num_global_features -- Number of additional features to be 
            #concatenated with the latent space observables to form the input to F.
            event = np.insert(event, 5, data[ievt][im]['boundary'][1]*np.ones(len(event)),axis=1)
            
            X_awk.append(event)
            trueenergy = mc_truth.item().get('true_energy') #TARGET
            Y += [trueenergy]
            
X_awk.end_list()

X = awk_to_numpy_padded(X_awk,fill_val,ncell_max,cell_axis)
Y = np.array(Y)
print("X shape =",np.shape(X),"([Images X Events][Cells][XYEDBB])") #X,Y,Energy,Depth,Boundary,Boundary
print("Y shape =",np.shape(Y),"[MC Truth Energy]")

N Events: 1000 N Images per Event: 400
ECAL X: (465,) + ECAL Y (465,) + ECAL E (465,) = (465, 3) [using np.c_]
HCAL X: (1, 75) + HCAL Y (1, 75) + HCAL E (1, 75) =  -> np.append to ECAL -> (603, 3)


---

In [ ]:
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split
import tensorflow as tf

import tensorflow as tf
tf.keras.utils.normalize(X, axis=-1, order=1)

Standardize input

In [ ]:
print(X[0][0][3])

In [ ]:
#ultimately want an array with 6 numbers, containing the mean of each (XYEDBB)
#probably easier to compute by hand [Images X Events][Cells][XYEDBB]
#X_train_new = (X_train-np.mean(X_train))/np.stdev(X_train)
import copy
X_nan = copy.deepcopy(X) # otherwise, 0 get's replaced with nan's...
X_nan[X_nan ==0] = np.nan

X_mean = np.nanmean(X_nan,axis=(0,1))
X_std = np.nanstd(X_nan,axis=(0,1))

print( np.shape(X_nan), np.shape(X_mean)[0], np.shape(X_std)[0] )
print( X_mean )
print( X_std )    
print(X_mean/X_std)
Test_X = (X-X_mean)/X_std
#Change original X dateset to be filled with NaN (or None), do this calculation, including stdev.

In [ ]:
#print(X[0][0][3])
#print(Test_X[0][0][3])
#Test_X = X

In [ ]:
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(Test_X, Y, val=0.2, test=0.3)

In [ ]:
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
output_act, output_dim = 'linear', 1
loss = 'mse' #mean-squared error
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes, 
          output_act=output_act, output_dim=output_dim, loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))
          #output_act=output_act, output_dim=output_dim, loss=loss)

In [ ]:
the_fit = pfn.fit(X_train, Y_train,
                  epochs=100,
                  batch_size=100,
                  validation_data=(X_val, Y_val),
                  verbose=1)

#myfit = pfn.fit...
#plot loss vs epoch #
#if val_loss goes up, overfitting. tf.earlystopping can prevent (call_back -- monitors or interevens during training)

In [ ]:
pfn.layers
mypreds = pfn.predict(X_test,batch_size=100)

In [ ]:
fig = plt.figure(figsize=(14,10))
plt.scatter(Y_test,mypreds)
plt.xlabel("Y Test [GeV]",fontsize=22)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Y Pred [GeV]",fontsize=22)
plt.title("Prediction vs. Test",fontsize=26)

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(the_fit.history['loss'])
plt.plot(the_fit.history['val_loss'])
plt.title('Model Loss vs. Epoch',fontsize=26)
plt.ylabel('Mean-Squared Error',fontsize=22)
plt.yscale('log')
plt.xlabel('epoch',fontsize=22)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([-1,101])
plt.legend(['train', 'validation'], loc='upper right',fontsize=22)
plt.show()

#print(Y_test)
#print(mypreds)